## News text summarization

In [ ]:
# API e0207a9648fb4f85af10c206189158e9


## import

In [ ]:
# общие
import requests
import numpy as np
import pandas as pd
import json
import csv

In [ ]:
# для сохранения файлов
from google.colab import files

In [ ]:
# для обработки текста
import nltk

from nltk.corpus import stopwords
from scipy.io import mmread
from nltk.tokenize import sent_tokenize
from gensim.summarization import keywords
from gensim.summarization.summarizer import summarize

from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Get and save news

In [ ]:
# url
url = 'https://newsapi.org/v2/top-headlines?country=us&apiKey=e0207a9648fb4f85af10c206189158e9'


In [ ]:
# запрашиваем адрес
get = requests.get(url).json()

In [ ]:
get

{'articles': [{'author': None,
   'content': 'LOS ANGELES Robert Forster, the handsome and omnipresent character actor who got a career resurgence and Oscar nomination for playing bail bondsman Max Cherry in Jackie Brown, died Friday. He was 78.\r\nPublicist Kathie Berlin said Forster died of brain cancer … [+4087 chars]',
   'description': 'Robert Forster, the actor who got a career resurgence for playing bail bondsman Max Cherry in “Jackie Brown,” has died. He was 78.',
   'publishedAt': '2019-10-12T08:30:00Z',
   'source': {'id': 'usa-today', 'name': 'USA Today'},
   'title': "Robert Forster, Oscar-nominated actor for 'Jackie Brown,' dies at 78 - USA TODAY",
   'url': 'https://www.usatoday.com/story/entertainment/2019/10/12/robert-forster-academy-award-nominee-jackie-brown-actor/3957049002/?utm_source=google&utm_medium=amp&utm_campaign=speakable',
   'urlToImage': 'https://www.gannett-cdn.com/presto/2019/10/12/USAT/1442e1dd-4f78-46a5-97f5-8be051c34497-AP_Obit_Robert_Forster.JPG?crop

In [ ]:
# метка
my_article = get['articles']

In [ ]:
# создаем список для новостей
list1 = []

# функция парсинга новостей
def get_news(lst):
  for ar in my_article:
    news_dict=dict()
    
    news_dict['author'] = ar['author']
    news_dict['published At'] = ar['publishedAt']
    news_dict['title'] = ar['title']
    news_dict['content'] = ar['content']
    news_dict['url'] = ar['url']
    
    lst.append(news_dict)
    #print(news_dict)
  return
    

In [ ]:
# получаем новости
get_news(list1)

In [ ]:
# ключи для датафрейма csv
csv_keys = ['author', 'published At', 'title', 'content', 'url']

In [ ]:
# создаем csv файл
with open('news.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=csv_keys)
    writer.writeheader()
    for item in list1:
        writer.writerow(item)

In [ ]:
# сохраняем парсинг
files.download('news.csv')

In [ ]:
data = pd.read_csv('news.csv')
data.head(3)

author  ...                                                url
0    NaN  ...  https://www.usatoday.com/story/entertainment/2...
1    NaN  ...  https://www.nytimes.com/2019/10/12/sports/eliu...
2    NaN  ...  https://www.nytimes.com/2019/10/12/world/asia/...

[3 rows x 5 columns]

## text processing

In [ ]:
# Разделение текстов на предложение

sentences = []
for i in data['content']:
  sentences.append(sent_tokenize(i))

In [ ]:
# проверим предложения
sentences[0:10]

[['LOS ANGELES Robert Forster, the handsome and omnipresent character actor who got a career resurgence and Oscar nomination for playing bail bondsman Max Cherry in Jackie Brown, died Friday.',
  'He was 78.',
  'Publicist Kathie Berlin said Forster died of brain cancer … [+4087 chars]'],
 ['What the event lacked in officially sanctioned gravitas it seemed determined to make up for with theater and grandiose proclamations.',
  'The run, organized by the petrochemical company INEOS, featured a cycle of hype and commercial buildup more reminiscent of a… [+620 chars]'],
 ['As Hagibis approached this week at one point the equivalent of a Category 5 hurricane, with 160 m.p.h.',
  'winds the Japanese authorities prepared for disruptions in the lives of millions.',
  'About 1.5 million people live below sea level in eastern parts of Tokyo, a… [+1361 chars]'],
 ['Despite the official announcement being on Tuesday, Best Buy in Canada has already listed the Pixel 4 and Pixel 4 XL for pre-order.'

In [ ]:
# flatten
sentences = [y for x in sentences for y in x ]

In [ ]:
sentences[]

['LOS ANGELES Robert Forster, the handsome and omnipresent character actor who got a career resurgence and Oscar nomination for playing bail bondsman Max Cherry in Jackie Brown, died Friday.',
 'He was 78.',
 'Publicist Kathie Berlin said Forster died of brain cancer … [+4087 chars]']

In [ ]:
# Удаляем пунктуацию и лишние знаки, приводим к нижнему регистру

clean_sent = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
clean_sent = [s.lower() for s in clean_sent]

In [ ]:
clean_sent[0]

'los angeles robert forster  the handsome and omnipresent character actor who got a career resurgence and oscar nomination for playing bail bondsman max cherry in jackie brown  died friday '

In [ ]:
# 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# stopwords
stop_words = stopwords.words('english')

In [ ]:
# проверяем кол-во
len(stop_words)

179

In [ ]:
# удаляем стоп слова из предложений
def remove_sw(sen):
  sen_new = " ".join([i for i in sen if i not in stop_words])
  return sen_new

clean_sent = [remove_sw(r.split()) for r in clean_sent]

In [ ]:
# качаем Glove
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-10-12 11:06:07--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-10-12 11:06:07--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-10-12 11:06:08--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
# Распаковываем Glove
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
# создаем словарь с векторами слов
word_embeddings = {}

f = open('glove.6B.100d.txt', encoding='utf-8')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
word_embeddings['cat']

array([ 0.23088  ,  0.28283  ,  0.6318   , -0.59411  , -0.58599  ,
        0.63255  ,  0.24402  , -0.14108  ,  0.060815 , -0.7898   ,
       -0.29102  ,  0.14287  ,  0.72274  ,  0.20428  ,  0.1407   ,
        0.98757  ,  0.52533  ,  0.097456 ,  0.8822   ,  0.51221  ,
        0.40204  ,  0.21169  , -0.013109 , -0.71616  ,  0.55387  ,
        1.1452   , -0.88044  , -0.50216  , -0.22814  ,  0.023885 ,
        0.1072   ,  0.083739 ,  0.55015  ,  0.58479  ,  0.75816  ,
        0.45706  , -0.28001  ,  0.25225  ,  0.68965  , -0.60972  ,
        0.19578  ,  0.044209 , -0.31136  , -0.68826  , -0.22721  ,
        0.46185  , -0.77162  ,  0.10208  ,  0.55636  ,  0.067417 ,
       -0.57207  ,  0.23735  ,  0.4717   ,  0.82765  , -0.29263  ,
       -1.3422   , -0.099277 ,  0.28139  ,  0.41604  ,  0.10583  ,
        0.62203  ,  0.89496  , -0.23446  ,  0.51349  ,  0.99379  ,
        1.1846   , -0.16364  ,  0.20653  ,  0.73854  ,  0.24059  ,
       -0.96473  ,  0.13481  , -0.0072484,  0.33016  , -0.1236

In [ ]:
sen_vectors = []


for i in clean_sent:
    
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sen_vectors.append(v)

In [ ]:
len(sen_vectors)

46

In [ ]:
sim_mat = np.zeros([len(sentences), len(sentences)])

In [ ]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sen_vectors[i].reshape(1,100), sen_vectors[j].reshape(1,100))[0,0]
      

In [ ]:
nx_graph = nx.from_numpy_array(sim_mat)

scores = nx.pagerank(nx_graph)

In [ ]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [ ]:
len(ranked_sentences)

46

In [ ]:
# Specify number of sentences to form the summary
sn = 10

# Generate summary
for i in range(sn):
    print(i,'=====\n',ranked_sentences[i][1])
    print('\n\nkewords:\n',keywords(ranked_sentences[i][1]))

0 =====
 Share this article
Companies in this article
Blizzard
On Friday evening, Blizzard Entertainment president J. Allen Brack released a statement in response to a week of ongoing criticism from players, developers, and even politicians over its decision to ban… [+3011 chars]


kewords:
 blizzard
brack
evening
1 =====
 If you've been looking at the Samsung Galaxy Note10+ but couldn't so far get yourself to actually order one, the Korean company is now running a promo that may just tip the scales for you.


kewords:
 galaxy
2 =====
 Despite the official announcement being on Tuesday, Best Buy in Canada has already listed the Pixel 4 and Pixel 4 XL for pre-order.


kewords:
 best
3 =====
 Theyve been trying to stop us for the past three years with a lot of crap, he stated matter-of-factly about his Democratic opponents on Capitol Hill.


kewords:
 stated
4 =====
 McAleenan became the fourth person to lead the agency under Trump in April after th… [+1455 chars]


kewords:
 
5 =====
 W